<a href="https://colab.research.google.com/github/12ljw/deep-learning-with-python-2nd/blob/main/Final_AI_Thumbnail_Selector(CLIP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training‑Free AI Thumbnail Selector (CLIP)

유튜브 **영상 제목(텍스트)** 과 **추출 프레임(이미지)** 간 의미 유사도를 계산해, *학습 없이* 베스트 썸네일을 추천.

**Pipeline**
1) ffmpeg로 10초 간격 프레임 추출  
2) CLIP 임베딩으로 제목–이미지 유사도 계산  
3) Top‑k 썸네일 추천 (+ 선택: 유사 프레임 중복 제거)


In [ ]:
!pip install pytube
!pip install ImageHash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
# YouTube 다운로드 설정
!pip install -q yt-dlp
import yt_dlp
import os
import shutil


YOUTUBE_URL = 'https://www.youtube.com/watch?v=dFO5O_jxUTk'
OUTPUT_MP4 = 'input.mp4'

ydl_opts = {
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
    'merge_output_format': 'mp4',
    'outtmpl': 'yt_download.%(ext)s',
    'noplaylist': True,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(YOUTUBE_URL, download=True)
    downloaded = ydl.prepare_filename(info)


if os.path.exists(OUTPUT_MP4):
    os.remove(OUTPUT_MP4)
shutil.move(downloaded if os.path.exists(downloaded) else 'yt_download.mp4', OUTPUT_MP4)
print('다운로드 완료:', OUTPUT_MP4)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.6 MB/s eta 0:00:00
[youtube] Extracting URL: https://www.youtube.com/watch?v=dFO5O_jxUTk
[youtube] dFO5O_jxUTk: Downloading webpage


[youtube] dFO5O_jxUTk: Downloading android sdkless player API JSON
[youtube] dFO5O_jxUTk: Downloading web safari player API JSON


[youtube] dFO5O_jxUTk: Downloading m3u8 information


[info] dFO5O_jxUTk: Downloading 1 format(s): 399+140-1
[download] Sleeping 5.00 seconds as required by the site...
[download] Destination: yt_download.f399.mp4
[download] 100% of  128.88MiB in 00:02:11 at 1005.43KiB/s
[download] Destination: yt_download.f140-1.m4a
[download] 100% of   14.47MiB in 00:00:09 at 1.59MiB/s   
[Merger] Merging formats into "yt_download.mp4"
Deleting original file yt_download.f140-1.m4a (pass -k to keep)
Deleting original file yt_download.f399.mp4 (pass -k to keep)
다운로드 완료: input.mp4


In [ ]:
# ffmpeg로 프레임 추출 (10초 간격)
import os

os.makedirs("frames", exist_ok=True)

!ffmpeg -i input.mp4 -vf fps=1/10 frames/frame_%05d.jpg

print("프레임 추출 완료.")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# pHash 중복 프레임 제거
from PIL import Image
import imagehash
import glob

def deduplicate_frames(frame_dir, threshold=5):
    frame_paths = sorted(glob.glob(os.path.join(frame_dir, "*.jpg")))
    kept = []
    hashes = []

    for path in frame_paths:
        img = Image.open(path).convert("RGB")
        h = imagehash.phash(img)

        if not any(abs(h - old_h) <= threshold for old_h in hashes):
            kept.append(path)
            hashes.append(h)

    print(f"Total frames: {len(frame_paths)}, Kept: {len(kept)}")
    return kept

frame_dir = "frames"
kept_frames = deduplicate_frames(frame_dir, threshold=5)

len(kept_frames)

Total frames: 94, Kept: 92


92

In [ ]:
!pip install ftfy regex tqdm git+https://github.com/openai/CLIP.git

import torch
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7qorpr6v
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7qorpr6v
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=d95cc2ecf3fb09cb990b19d15aa93fd4684bec0ef30636978ec609d85b2d8452
  Stored in directory: /tmp/pip-ephem-wheel-cache-scs7egie/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip
device: cpu


100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 97.9MiB/s]


In [ ]:
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

# 얼굴 검출 algorithm (Haar cascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def brightness_score(img_bgr):
    # 이미지 전체 밝기 (0~1)
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    return hsv[..., 2].mean() / 255.0

def complexity_score(img_bgr):
    # 엣지 양으로 복잡도 측정 (0~1, 높을수록 복잡)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return edges.mean() / 255.0

def center_focus_score(img_bgr):
    # 화면 중앙부에 정보(edge)가 얼마나 모여 있는지 (0~1)
    h, w, _ = img_bgr.shape
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200) / 255.0

    ch, cw = int(h * 0.5), int(w * 0.5)
    y1, y2 = (h - ch) // 2, (h + ch) // 2
    x1, x2 = (w - cw) // 2, (w + cw) // 2

    center_region = edges[y1:y2, x1:x2]
    return center_region.mean()

def face_score(img_bgr):
    # 얼굴이 잘 보일수록 score 높음
    # 얼굴 영역 / 전체 영역 비율을 0~1로 스케일링
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(40, 40)
    )

    if len(faces) == 0:
        return 0.0

    h, w, _ = img_bgr.shape
    img_area = h * w

    face_area_sum = 0
    for (x, y, fw, fh) in faces:
        face_area_sum += fw * fh

    ratio = face_area_sum / img_area   # 얼굴 영역 비율
    return float(min(ratio * 3.0, 1.0))   # 0~1 사이로 조정

In [ ]:
def score_frames_clip_only(frame_paths, title_text):
    # Baseline : CLIP 유사도만 사용하는 Frame Score
    with torch.no_grad():
        text_tokens = clip.tokenize([title_text]).to(device)
        text_feat = clip_model.encode_text(text_tokens)
        text_feat /= text_feat.norm(dim=-1, keepdim=True)

    results = []

    for path in tqdm(frame_paths, desc="CLIP-only scoring"):
        pil = Image.open(path).convert("RGB")
        img_tensor = clip_preprocess(pil).unsqueeze(0).to(device)

        with torch.no_grad():
            img_feat = clip_model.encode_image(img_tensor)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            sim = (img_feat @ text_feat.T).item()

        results.append({
            "path": path,
            "clip": sim
        })

    return sorted(results, key=lambda x: x["clip"], reverse=True)


def score_frames_clip_quality(frame_paths, title_text):
    # CLIP + 밝기 + 중앙부 집중도 + 복잡도 + 얼굴 score 를 합친 Thumbnail Score
    with torch.no_grad():
        text_tokens = clip.tokenize([title_text]).to(device)
        text_feat = clip_model.encode_text(text_tokens)
        text_feat /= text_feat.norm(dim=-1, keepdim=True)

    results = []

    for path in tqdm(frame_paths, desc="CLIP+Quality scoring"):
        pil = Image.open(path).convert("RGB")
        img_tensor = clip_preprocess(pil).unsqueeze(0).to(device)
        img_bgr = cv2.imread(path)

        # 1) CLIP 유사도
        with torch.no_grad():
            img_feat = clip_model.encode_image(img_tensor)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            sim = (img_feat @ text_feat.T).item()

        # 2) 시각적 quality score
        b  = brightness_score(img_bgr)
        c  = complexity_score(img_bgr)
        f  = center_focus_score(img_bgr)
        fs = face_score(img_bgr)

        # 3) 가중치
        alpha = 0.55  # CLIP similarity
        beta  = 0.10  # brightness
        gamma = 0.10  # center focus
        delta = 0.10  # complexity
        eps   = 0.15  # face score

        thumbnail_score = (
            alpha * sim +
            beta  * b +
            gamma * f -
            delta * c +
            eps   * fs
        )

        results.append({
            "path": path,
            "clip": sim,
            "brightness": b,
            "complexity": c,
            "focus": f,
            "face": fs,
            "score": thumbnail_score
        })

    return sorted(results, key=lambda x: x["score"], reverse=True)

In [ ]:
def apply_face_focus_blur(img_bgr, ksize=51):
    # 프레임 내에
    # 1) 얼굴이 있으면: 얼굴을 감싸는 영역만 강조, 나머지 블러처리
    # 2) 얼굴이 없으면: 화면 중앙 기준으로 블러처리
    h, w, _ = img_bgr.shape
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(40, 40)
    )

    if len(faces) == 0:
        # 프레임 내에 얼굴이 없으면 프레임 중앙을 기준으로 blurring
        ch, cw = int(h * 0.5), int(w * 0.5)
        y1, y2 = (h - ch)//2, (h + ch)//2
        x1, x2 = (w - cw)//2, (w + cw)//2

        blurred = cv2.GaussianBlur(img_bgr, (ksize, ksize), 0)
        out = blurred.copy()
        out[y1:y2, x1:x2] = img_bgr[y1:y2, x1:x2]
        return out

    # 프레임 내에 얼굴이 여러 개일 경우: 전체를 감싸는 큰 박스 생성
    x_min = min([x for (x, y, fw, fh) in faces])
    y_min = min([y for (x, y, fw, fh) in faces])
    x_max = max([x + fw for (x, y, fw, fh) in faces])
    y_max = max([y + fh for (x, y, fw, fh) in faces])

    margin_x = int(0.1 * w)
    margin_y = int(0.1 * h)
    x1 = max(0, x_min - margin_x)
    y1 = max(0, y_min - margin_y)
    x2 = min(w, x_max + margin_x)
    y2 = min(h, y_max + margin_y)

    blurred = cv2.GaussianBlur(img_bgr, (ksize, ksize), 0)
    out = blurred.copy()
    out[y1:y2, x1:x2] = img_bgr[y1:y2, x1:x2]
    return out

In [ ]:
import os

TITLE = "Choo Sung Hoon Living in a Rented Room in SHIHO's House"

# 1) Baseline & Thumbnail Score 계산
clip_only = score_frames_clip_only(kept_frames, TITLE)
clip_qual = score_frames_clip_quality(kept_frames, TITLE)

print("=== CLIP-only Top 3 ===")
for i, s in enumerate(clip_only[:3], start=1):
    print(f"[{i}] {os.path.basename(s['path'])}  |  CLIP={s['clip']:.3f}")

print("\n=== CLIP+Quality(+Face) Top 3 ===")
for i, s in enumerate(clip_qual[:3], start=1):
    print(
        f"[{i}] {os.path.basename(s['path'])}  |  Score={s['score']:.3f}  "
        f"(CLIP={s['clip']:.3f}, B={s['brightness']:.3f}, "
        f"F={s['focus']:.3f}, C={s['complexity']:.3f}, Face={s['face']:.3f})"
    )

CLIP+Quality scoring: 100%|██████████| 92/92 [01:26<00:00,  1.07it/s]

=== CLIP-only Top 3 ===
[1] frame_00047.jpg  |  CLIP=0.361
[2] frame_00079.jpg  |  CLIP=0.355
[3] frame_00068.jpg  |  CLIP=0.333

=== CLIP+Quality(+Face) Top 3 ===
[1] frame_00037.jpg  |  Score=0.278  (CLIP=0.318, B=0.445, F=0.001, C=0.006, Face=0.397)
[2] frame_00079.jpg  |  Score=0.277  (CLIP=0.355, B=0.689, F=0.013, C=0.007, Face=0.083)
[3] frame_00024.jpg  |  Score=0.270  (CLIP=0.322, B=0.634, F=0.029, C=0.033, Face=0.198)


In [ ]:
os.makedirs("results", exist_ok=True)

# 1) Baseline(CLIP-only) Top-1 Thumbnail
best_clip = clip_only[0]
img_clip = cv2.imread(best_clip["path"])
cv2.imwrite("results/best_clip_only.jpg", img_clip)

# 2) 개선 버전 Top-1 (원본 + 얼굴 중심 블러)
best_qual = clip_qual[0]
img_qual = cv2.imread(best_qual["path"])
img_qual_face = apply_face_focus_blur(img_qual, ksize=51)

cv2.imwrite("results/best_quality_raw.jpg", img_qual)
cv2.imwrite("results/best_quality_face_focus.jpg", img_qual_face)

print("저장된 파일:")
print("- results/best_clip_only.jpg")
print("- results/best_quality_raw.jpg")
print("- results/best_quality_face_focus.jpg")

저장된 파일:
- results/best_clip_only.jpg
- results/best_quality_raw.jpg
- results/best_quality_face_focus.jpg
